<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/49_5_Speech_Technologies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Установка библиотек
%%capture
!pip install SpeechRecognition pydub
!apt install ffmpeg

In [2]:
# Импорт библиотек
import os
import speech_recognition as sr
from pydub import AudioSegment
from pydub.generators import WhiteNoise
import numpy as np
from sklearn.metrics import accuracy_score

In [3]:
# Загрузка аудиофайлов
from google.colab import files
uploaded = files.upload()

Saving utro.mp3 to utro.mp3
Saving novoe.mp3 to novoe.mp3
Saving ne-mogu.mp3 to ne-mogu.mp3
Saving jenskie-golosovye.mp3 to jenskie-golosovye.mp3
Saving dobrogo-vremni-sutok.mp3 to dobrogo-vremni-sutok.mp3


In [4]:
# Функция конвертации mp3 в wav
def convert_mp3_to_wav(input_mp3_path, output_wav_path=None, sample_rate=16000):

    # Загружаем MP3 файл
    audio = AudioSegment.from_mp3(input_mp3_path)

    # Устанавливаем частоту дискретизации
    audio = audio.set_frame_rate(sample_rate)

    # Экспортируем в WAV
    audio.export(output_wav_path, format="wav")
    print(f"Файл успешно конвертирован: {output_wav_path}")
    return output_wav_path

In [5]:
# Функция добавления шума
def add_noise_to_audio(input_path, output_path=None, noise_level=0.9):

    # Загружаем аудиофайл
    audio = AudioSegment.from_file(input_path)

    # Генерируем белый шум той же длительности, что и аудио
    noise = WhiteNoise().to_audio_segment(duration=len(audio))

    # Накладываем шум на оригинальное аудио
    noisy_audio = audio.overlay(noise, gain_during_overlay=noise_level*20)

    # Если выходной путь не указан, добавляем '_noisy' к имени файла
    if output_path is None:
        base, ext = os.path.splitext(input_path)
        output_path = f"{base}_noisy{ext}"

    # Сохраняем результат
    noisy_audio.export(output_path, format=output_path.split('.')[-1])

    print(f"Файл с шумом успешно создан: {output_path}")
    return output_path

In [6]:
# Функция транскрибации аудио
def transcribe_audio(filename):

    r = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ru-RU")
        return text

In [7]:
# Конвертирум mp3 в wav
folder_path = '/content/'

# Создаем папку для чистых аудиофайлов
os.makedirs("clean", exist_ok=True)

# Получаем список всех файлов и папок
all_items = os.listdir(folder_path)

# Фильтруем только файлы (исключаем подпапки)
clean_files = [item for item in all_items if os.path.isfile(os.path.join(folder_path, item))]
print(clean_files)

for file in clean_files:
    convert_mp3_to_wav(file, f"clean/{file[:-4]}_clean.wav")

['jenskie-golosovye.mp3', 'ne-mogu.mp3', 'utro.mp3', 'novoe.mp3', 'dobrogo-vremni-sutok.mp3']
Файл успешно конвертирован: clean/jenskie-golosovye_clean.wav
Файл успешно конвертирован: clean/ne-mogu_clean.wav
Файл успешно конвертирован: clean/utro_clean.wav
Файл успешно конвертирован: clean/novoe_clean.wav
Файл успешно конвертирован: clean/dobrogo-vremni-sutok_clean.wav


In [8]:
# Добавляем шум к записям
folder_path = '/content/clean/'

# Создаем папку для шумных аудиофайлов
os.makedirs("noise", exist_ok=True)

# Получаем список всех файлов и папок
all_items = os.listdir(folder_path)

# Фильтруем только файлы (исключаем подпапки)
noise_files = [item for item in all_items if os.path.isfile(os.path.join(folder_path, item))]
print(noise_files)

for file in noise_files:
    add_noise_to_audio(f"{folder_path}{file}", f"noise/{file[:-10]}_noise.wav")

['utro_clean.wav', 'dobrogo-vremni-sutok_clean.wav', 'novoe_clean.wav', 'jenskie-golosovye_clean.wav', 'ne-mogu_clean.wav']
Файл с шумом успешно создан: noise/utro_noise.wav
Файл с шумом успешно создан: noise/dobrogo-vremni-sutok_noise.wav
Файл с шумом успешно создан: noise/novoe_noise.wav
Файл с шумом успешно создан: noise/jenskie-golosovye_noise.wav
Файл с шумом успешно создан: noise/ne-mogu_noise.wav


In [9]:
# Список с оригинальным текстом и для сохрения рез-та
phrases = ["Доброго времени суток",
           "В чем ты сейчас одет",
           "Блин к сожалению не могу",
           "Для меня это что-то новое",
           "Доброе утро"]

results = []

In [10]:
# Проводим транкрибацию и добавляем в словарь
clean_folder_path = '/content/clean/'
noise_folder_path = '/content/noise/'

# Получаем список файлов и папок
clean_contents = sorted(os.listdir(clean_folder_path))
noise_contents = sorted(os.listdir(noise_folder_path))

for i in range(5):
    clean_text = transcribe_audio(f"{clean_folder_path}{clean_contents[i]}")
    noisy_text = transcribe_audio(f"{noise_folder_path}{noise_contents[i]}")
    original_text = phrases[i].lower()

    results.append({
      "sample": i,
      "original": original_text,
      "clean_transcription": clean_text.lower() if clean_text else "",
      "noisy_transcription": noisy_text.lower() if noisy_text else "",
  })

In [11]:
# Проверяем порядок чтения в папке
print(clean_contents)
print(noise_contents)

['dobrogo-vremni-sutok_clean.wav', 'jenskie-golosovye_clean.wav', 'ne-mogu_clean.wav', 'novoe_clean.wav', 'utro_clean.wav']
['dobrogo-vremni-sutok_noise.wav', 'jenskie-golosovye_noise.wav', 'ne-mogu_noise.wav', 'novoe_noise.wav', 'utro_noise.wav']


In [12]:
# Вывод результата
for res in results:
    print(res)

{'sample': 0, 'original': 'доброго времени суток', 'clean_transcription': 'доброго времени су', 'noisy_transcription': 'доброго времени'}
{'sample': 1, 'original': 'в чем ты сейчас одет', 'clean_transcription': 'в чём ты сейчас', 'noisy_transcription': 'как пишется'}
{'sample': 2, 'original': 'блин к сожалению не могу', 'clean_transcription': 'блин к сожалению не могу', 'noisy_transcription': 'население'}
{'sample': 3, 'original': 'для меня это что-то новое', 'clean_transcription': 'для меня это что-то новое', 'noisy_transcription': 'для меня это что-то новое'}
{'sample': 4, 'original': 'доброе утро', 'clean_transcription': 'доброе утро', 'noisy_transcription': 'доброе'}


In [13]:
# Метрики и сравнение
def calculate_metrics(original, transcribed):

    original_words = original.split()
    transcribed_words = transcribed.split()

    # Точность на уровне слов (Word Accuracy)
    correct_words = sum(1 for ow, tw in zip(original_words, transcribed_words) if ow == tw)
    word_accuracy = correct_words / len(original_words) if original_words else 0

    # Точность на уровне символов (Character Accuracy)
    original_chars = list(original.replace(" ", ""))
    transcribed_chars = list(transcribed.replace(" ", ""))
    correct_chars = sum(1 for oc, tc in zip(original_chars, transcribed_chars) if oc == tc)
    char_accuracy = correct_chars / len(original_chars) if original_chars else 0

    # WER (Word Error Rate) - упрощенный расчет
    errors = sum(1 for ow, tw in zip(original_words, transcribed_words) if ow != tw)
    wer = errors / len(original_words) if original_words else 1

    return word_accuracy, char_accuracy, wer

# Вычисляем метрики для каждого образца
for res in results:
    # Для чистого аудио
    wa_clean, ca_clean, wer_clean = calculate_metrics(res["original"], res["clean_transcription"])

    # Для шумного аудио
    wa_noisy, ca_noisy, wer_noisy = calculate_metrics(res["original"], res["noisy_transcription"])

    res.update({
        "word_accuracy_clean": wa_clean,
        "char_accuracy_clean": ca_clean,
        "wer_clean": wer_clean,
        "word_accuracy_noisy": wa_noisy,
        "char_accuracy_noisy": ca_noisy,
        "wer_noisy": wer_noisy,
    })

In [14]:
# Вывод результатов
print("{:<10} {:<25} {:<25} {:<15} {:<15} {:<15} {:<15}".format(
    "Образец", "Оригинальный текст", "Чистая транскрипция", "WA чист", "WA шум", "CA чист", "CA шум"))
print("-" * 110)

for res in results:
    print("{:<10} {:<25} {:<25} {:<15.2f} {:<15.2f} {:<15.2f} {:<15.2f}".format(
        res["sample"],
        res["original"][:20] + "..." if len(res["original"]) > 20 else res["original"],
        (res["clean_transcription"][:20] + "..." if len(res["clean_transcription"]) > 20 else res["clean_transcription"]) if res["clean_transcription"] else "-",
        res["word_accuracy_clean"],
        res["word_accuracy_noisy"],
        res["char_accuracy_clean"],
        res["char_accuracy_noisy"]))

Образец    Оригинальный текст        Чистая транскрипция       WA чист         WA шум          CA чист         CA шум         
--------------------------------------------------------------------------------------------------------------
0          доброго времени суто...   доброго времени су        0.67            0.67            0.84            0.74           
1          в чем ты сейчас одет      в чём ты сейчас           0.60            0.00            0.69            0.00           
2          блин к сожалению не ...   блин к сожалению не ...   1.00            0.00            1.00            0.00           
3          для меня это что-то ...   для меня это что-то ...   1.00            1.00            1.00            1.00           
4          доброе утро               доброе утро               1.00            0.50            1.00            0.60           


In [15]:
# Средние значения метрик
avg_wa_clean = np.mean([res["word_accuracy_clean"] for res in results])
avg_wa_noisy = np.mean([res["word_accuracy_noisy"] for res in results])
avg_ca_clean = np.mean([res["char_accuracy_clean"] for res in results])
avg_ca_noisy = np.mean([res["char_accuracy_noisy"] for res in results])
avg_wer_clean = np.mean([res["wer_clean"] for res in results])
avg_wer_noisy = np.mean([res["wer_noisy"] for res in results])

print("\nСредние значения метрик:")
print(f"Точность на уровне слов (чистое аудио): {avg_wa_clean:.2f}")
print(f"Точность на уровне слов (шумное аудио): {avg_wa_noisy:.2f}")
print(f"Точность на уровне символов (чистое аудио): {avg_ca_clean:.2f}")
print(f"Точность на уровне символов (шумное аудио): {avg_ca_noisy:.2f}")
print(f"WER (чистое аудио): {avg_wer_clean:.2f}")
print(f"WER (шумное аудио): {avg_wer_noisy:.2f}")


Средние значения метрик:
Точность на уровне слов (чистое аудио): 0.85
Точность на уровне слов (шумное аудио): 0.43
Точность на уровне символов (чистое аудио): 0.91
Точность на уровне символов (шумное аудио): 0.47
WER (чистое аудио): 0.11
WER (шумное аудио): 0.12
